In [1]:
import openai
import os

# Expects OPENAI_API_KEY in env variables 
# For GitHub Codespaces: set this as Codespaces secret => shows up as env var in OS
# For Docker Desktop: create a .env file (and .gitignore it explicitly to be safe) => shows up as env var from load_dotenv
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

# Note that we can set different env variables to different OPENAI keys and just map the right one to openai.api_key here
# Example: have both OPENAI_API_KEY (for OpenAI) and AOAI_API_KEY (for Azure OpenAI) as options 
openai.api_key  = os.getenv('OPENAI_API_KEY')

# Print Environment Variables
#for var in os.environ:
#    print(f"{var}: {os.environ[var]}")

def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.7, # this is the degree of randomness of the model's output
        max_tokens=1024
    )
    return response.choices[0].message.content

In [12]:
name=""
email=""
## Set the primary content or simple prompt text here
text = f"""
from flask import Flask, request
from flask_wtf import FlaskForm
from wtforms import StringField, SubmitField
from wtforms.validators import DataRequired, Length, Email

app = Flask(__name__)
app.config['SECRET_KEY'] = 'secret_key'

class HelloForm(FlaskForm):
    name = StringField('Name', validators=[DataRequired(), Length(min=3)])
    email = StringField('Email', validators=[DataRequired(), Email()])
    submit = SubmitField('Submit')

@app.route('/', methods=['GET', 'POST'])
def hello():
    form = HelloForm()
    if form.validate_on_submit():
        name = form.name.data
        email = form.email.data
        return f'Hello, {name} ({email})!'
    return form.render_template()
"""

## This uses a template that embeds the text 
## allowing you to add additional content like instructions, cues, examples
prompt = f"""
When I run the code below, I have the error : AttributeError: 'HelloForm' object has no attribute 'render_template'
Can you correct the code?
```{text}```
"""



In [13]:
## Run the prompt
response = get_completion(prompt)
print(response)

To fix the error, you need to change the line `return form.render_template()` to `return render_template('hello.html', form=form)`. This will render the template called "hello.html" and pass the `form` object to the template.

Here's the corrected code:

```python
from flask import Flask, render_template, request
from flask_wtf import FlaskForm
from wtforms import StringField, SubmitField
from wtforms.validators import DataRequired, Length, Email

app = Flask(__name__)
app.config['SECRET_KEY'] = 'secret_key'

class HelloForm(FlaskForm):
    name = StringField('Name', validators=[DataRequired(), Length(min=3)])
    email = StringField('Email', validators=[DataRequired(), Email()])
    submit = SubmitField('Submit')

@app.route('/', methods=['GET', 'POST'])
def hello():
    form = HelloForm()
    if form.validate_on_submit():
        name = form.name.data
        email = form.email.data
        return f'Hello, {name}!'
    return render_template('hello.html', form=form)
```

Make sure yo